# 📘 Galileo + OpenTelemetry Tracing Guide (Python)

This notebook demonstrates how to send OpenTelemetry traces to Galileo using OpenInference.

🔗 See full documentation [here](https://v2galileo.mintlify.app/guides/logs/otel)

In [ ]:
pip install -U \
    llama-index \
    openinference-instrumentation-llama-index \
    opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-api \
    "opentelemetry-proto>=1.12.0" -q

In [ ]:
GALILEO_API_KEY = "YOUR_API_KEY"
PROJECT_NAME = "YOUR-PROJECT-NAME"
LOG_STREAM_NAME = "my_log_stream"



## 🔐 Authentication and Header Setup

In [ ]:
import os

headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": PROJECT_NAME,
    "logstream": LOG_STREAM_NAME,
}
# Format headers as a comma-separated key=value string
# Example: Galileo-API-Key=abc123,project_name=my_project,log_stream_name=my_run
os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'] = ",".join([f"{k}={v}" for k, v in headers.items()])

## ⚙️ OpenTelemetry Setup

In [ ]:

from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

endpoint = "https://app.galileo.ai/api/galileo/otel/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

OTEL tracing initialized.


In [ ]:


from llama_index.agent.openai import OpenAIAgent
from llama_index.core import Settings
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
agent = OpenAIAgent.from_tools([multiply_tool, add_tool])
Settings.llm = OpenAI(model="gpt-3.5-turbo")

response = agent.query("What is (121 * 3) + 42?")
print(response)